In [21]:
from models.resnet.resnet_se_34v2 import ResNetSE34V2
from torchaudio.transforms import MelSpectrogram
from torchaudio.functional import amplitude_to_DB
import torch
import torchaudio
import librosa
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import yaml
import random

In [22]:
DATASET_PATH = "IEMOCAP_full_release"
PATH = os.path.join(os.getcwd(), DATASET_PATH)

In [109]:
data = []
spk_M = 0
spk_F = 0

for i, subdir in enumerate(os.listdir(PATH)):


    if (subdir[0] != '.'):
        #for j in random.sample(range(0, 20), 2):
        #for i, file in enumerate(os.listdir(os.path.join(PATH, subdir)+'/sentences/wav')):
        file = os.listdir(os.path.join(PATH, subdir)+'/sentences/wav')
        file_M = [path for path in file if path[5] == 'M']
        file_F = [path for path in file if path[5] == 'F']

        #Choose 2 random file in a directory 
        for j in random.sample(range(0, 10), 2):
            
      
            file2 = os.listdir(os.path.join(PATH, subdir)+'/sentences/wav/'+file_M[j]) 

            file2_M = [path for path in file2 if path[-8] == 'M']
            file2_F = [path for path in file2 if path[-8] == 'F']

            for k in random.sample(range(0, 10), 5):


                sample_M = os.path.join(PATH, subdir)+'/sentences/wav/'+file_M[j] +'/'+ file2_M[k]
                data.append([sample_M, "Spk_"+str(2*int(subdir[-1])-1)])
                sample_F = os.path.join(PATH, subdir)+'/sentences/wav/'+file_M[j] +'/'+ file2_F[k]
                data.append([sample_F, "Spk_"+str(2*int(subdir[-1]))])


            file2 = os.listdir(os.path.join(PATH, subdir)+'/sentences/wav/'+file_F[j])

            file2_M = [path for path in file2 if path[-8] == 'M']
            file2_F = [path for path in file2 if path[-8] == 'F']

            for k in random.sample(range(0, 10), 5):


                sample_M = os.path.join(PATH, subdir)+'/sentences/wav/'+file_F[j] +'/'+ file2_M[k]
                data.append([sample_M, "Spk_"+str(2*int(subdir[-1])-1)])
                sample_F = os.path.join(PATH, subdir)+'/sentences/wav/'+file_F[j] +'/'+ file2_F[k]
                data.append([sample_F, "Spk_"+str(2*int(subdir[-1]))])

In [110]:
print(len(data))

200


In [111]:
num_spk = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 
           7:0, 8:0, 9:0}

for i in data : 
    num_spk[int(i[1][-1])] += 1

print(num_spk)

{0: 20, 1: 20, 2: 20, 3: 20, 4: 20, 5: 20, 6: 20, 7: 20, 8: 20, 9: 20}


In [78]:
def load_audio(file):
    EPS = 1e-8
    s, _ = librosa.load(file, sr=16000)
    amax = np.max(np.abs(s))
    factor = 1.0 / (amax + EPS)
    s = s * factor
    return s

In [79]:
with open('./models/resnet/config.yaml') as f:
    config = yaml.safe_load(f)

sd = torch.load('./models/weigths/resnetse34_epoch92_eer0.00931.pth')
model = ResNetSE34V2(nOut=256, n_mels= 80)
model.load_state_dict(sd)
model.eval()
torch.set_grad_enabled(False)

transform = MelSpectrogram(
    sample_rate= 16000,
    n_fft= 512,
    win_length= 400,
    hop_length= 160,
    window_fn=torch.hamming_window,
    n_mels= 80,
    f_min=20,
    f_max=7600,
    norm='slaney')


def embed_inference(audio_path, transform= transform, model = model):
    s = load_audio(audio_path)
    #print(type(s))
    x = torch.from_numpy(s[None, :])
    x = transform(x)
    x = amplitude_to_DB(
        x, multiplier=10, amin= 1e-5, db_multiplier=0, top_db=75)

    feature = model(x[:, None, :, :])
    feature = torch.nn.functional.normalize(feature)
    return(feature,s)

Embedding size is 256, encoder SAP.


# PROCESS THE TEST

In [26]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

In [27]:
from speechbrain.pretrained import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")


In [98]:
from modelscope.models import Model
from modelscope.pipelines import pipeline

model = Model.from_pretrained('damo/speech_campplus_sv_en_voxceleb_16k', device='cpu')


MODEL CONFIG :  modelscope/1.9.2; python/3.8.18; session_id/61c54b3edbd54e49a8d63cd67ff19afc; platform/macOS-13.3-x86_64-i386-64bit; processor/i386; env/custom; user/unknown; invoked_by/from_pretrained


2023-10-11 14:45:13,697 - modelscope - INFO - Model revision not specified, use revision: v1.0.2
2023-10-11 14:45:14,472 - modelscope - INFO - initialize model from /Users/hugo/.cache/modelscope/hub/damo/speech_campplus_sv_en_voxceleb_16k


In [29]:
sim = torch.nn.CosineSimilarity(dim=1, eps=1e-8)

In [102]:
print(data)

[['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_M030.wav', 'Spk_7'], ['Ses04M_impro02_F011.wav', 'Spk_8'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_M026.wav', 'Spk_7'], ['Ses04M_impro02_F017.wav', 'Spk_8'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_M025.wav', 'Spk_7'], ['Ses04M_impro02_F006.wav', 'Spk_8'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_M032.wav', 'Spk_7'], ['Ses04M_impro02_F003.wav', 'Spk_8'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_M018.wav', 'S

In [115]:

compar_tab = pd.DataFrame(columns = ['spk_1', 'spk_2', 'duration_1', 'duration_2', 'score_classic', 'score_CAM++', 'TDN_score','path_1', 'path_2'])
already_done = []
with tqdm(total=len(data)*len(data)) as pbar:
    for sample_1 in data :
        spk_1 = sample_1[1]
        ref_embed, _ = embed_inference(sample_1[0])

        audio_1, _ = torchaudio.load(sample_1[0])
        for sample_2 in data :

            if {sample_1[0],sample_2[0]} not in already_done :
                print('test')
                spk_2 = sample_2[1]

                
                embed, _ = embed_inference(sample_2[0])
                audio_2, _ = torchaudio.load(sample_2[0])
                loss_val = sim(ref_embed, embed)

                #print(audio_1.shape, audio_1)
                embeddings = model(audio_1)
                embeddings2 = model(audio_2)
                cam_score_model = sim(embeddings, embeddings2)
                



                TDN_score = verification.verify_files(sample_1[0], sample_2[0]) # Different Speakers
                compar_tab.loc[len(compar_tab)] = [spk_1, spk_2, len(audio_1)/16000, len(audio_2)/16000, loss_val.item(),cam_score_model, TDN_score, sample_1[0], sample_2[0]]

                already_done.append({sample_1[0],sample_2[0]})
                #print(label, val)
            pbar.update(1)
       

           
compar_tab.to_csv('compar_tab_2.csv', index=False)

  0%|          | 0/40000 [00:00<?, ?it/s]

test


  0%|          | 1/40000 [00:02<28:15:16,  2.54s/it]

test


  0%|          | 2/40000 [00:03<19:27:56,  1.75s/it]

...

test


  0%|          | 17/40000 [00:29<19:51:12,  1.79s/it]

test


  0%|          | 17/40000 [00:30<19:47:19,  1.78s/it]


KeyboardInterrupt: 

In [47]:
import numpy as np
from sklearn.metrics import roc_curve, auc

def calculate_eer_and_fpr_fnr(scores, labels):
    fpr, tpr, thresholds = roc_curve(labels, scores)
    fnr = 1 - tpr

    eer_threshold = thresholds[np.nanargmin(np.abs(fnr - fpr))]
    eer = fnr[np.nanargmin(np.abs(fnr - fpr))]

    fpp = np.sum((scores >= eer_threshold) & (labels == 0))
    fnp = np.sum((scores < eer_threshold) & (labels == 1))
    total_negatives = np.sum(labels == 0)
    total_positives = np.sum(labels == 1)

    print(labels)
    print(scores)
    return eer, fpp, fnp, total_positives, total_negatives

# Replace 'verification_scores' and 'verification_labels' with your actual data


# print(f"Equal Error Rate (EER): {eer * 100:.2f}%")
# print(f"False Positive Predictions (FPP): {fpp}")
# print(f"False Negative Predictions (FNP): {fnp}")
# print(f"Total Positives: {total_positives}")
# print(f"Total Negatives: {total_negatives}")


******************************************************************
********************** CHOSEN TRESHOLD :  0.2  **********************


IndexError: list index out of range

In [37]:
text = 'tensor([1.0000])'

In [40]:
# How to change a str written like {'score':1.0 , 'text':'hello'} to a dict
def str_to_dict_cam(input : str):
    input = input.replace(' ', '')
    input = input.replace('tensor', '')
    input = input.replace('(', '')
    input = input.replace(')', '')
    input = input.replace("[", '')
    input = input.replace("]", '')


    return input

In [41]:
print(str_to_dict_cam(text))

1.0000


In [33]:
def str_to_dict_TDN(input : str):
    input = input.replace('tensor', '')
    input = input.replace(' ', '')
    input = input.replace('{', '')
    input = input.replace('}', '')
    input = input.replace("'", '')
    input = input.replace("(", '')
    input = input.replace(")", '')
    input = input.replace('[', '')
    input = input.replace(']', '')

    output = input.split(',')

    return output

In [52]:
df = pd.read_csv('compar_tab.csv')

false_positive_classic = 0
false_positive_CAM = 0
false_negative_classic = 0
false_negative_CAM = 0
false_positive_TDN = 0
false_negative_TDN = 0
positive = 0
negative = 0
treshold = 0.3

for treshold in [0.2,0.3,0.4,0.5,0.6,0.7] :
    print('********************** CHOSEN TRESHOLD : ', treshold, ' **********************')
    for row, line in df.iterrows():
        score_classic = line['score_classic']
    
        CAM = str_to_dict_cam(line['score_CAM++'])
        score_CAM = float(CAM)
        
        TDN = str_to_dict_TDN(line['TDN_score'])
        score_TDN = float(TDN[0])

        if line['spk_1'] == line['spk_2'] :
            positive += 1
            if score_classic < treshold :
                false_positive_classic += 1
            if score_CAM < treshold :
                false_positive_CAM += 1
            if score_TDN < treshold :
                false_positive_TDN += 1
        else :
            negative += 1
            if score_classic > treshold :
                false_negative_classic += 1
            if score_CAM > treshold :
                false_negative_CAM += 1
            if score_TDN > treshold :
                false_negative_TDN += 1

    print(f'For RESNET : False positive : {false_positive_classic} False negative : {false_negative_classic}')
    print(f'For CAM++ : False positive : {false_positive_CAM} False negative : {false_negative_CAM}')
    print(f'For TDN : False positive : {false_positive_TDN} False negative : {false_negative_TDN}')
    print(f'For RESNET : EER Accuracy : {100*((false_negative_classic/negative)+(false_positive_classic/positive))/2}')
    print(f'For CAM++ : EER Accuracy : {100*((false_negative_CAM/negative)+(false_positive_CAM/positive))/2}')   
    print(f'For TDN : EER Accuracy : {100*((false_negative_TDN/negative)+(false_positive_TDN/positive))/2}')

    false_positive_classic = 0
    false_positive_CAM = 0
    false_negative_classic = 0
    false_negative_CAM = 0
    false_positive_TDN = 0
    false_negative_TDN = 0
    positive = 0
    negative = 0

********************** CHOSEN TRESHOLD :  0.2  **********************
For RESNET : False positive : 22 False negative : 142
For CAM++ : False positive : 18 False negative : 84
For TDN : False positive : 20 False negative : 32
For RESNET : EER Accuracy : 17.903285345145807
For CAM++ : EER Accuracy : 12.56921373200443
For TDN : EER Accuracy : 10.003691399040235
********************** CHOSEN TRESHOLD :  0.3  **********************
For RESNET : False positive : 32 False negative : 36
For CAM++ : False positive : 30 False negative : 10
For TDN : False positive : 36 False negative : 6
For RESNET : EER Accuracy : 15.023994093761534
For CAM++ : EER Accuracy : 12.550756736803248
For TDN : EER Accuracy : 14.673311184939092
********************** CHOSEN TRESHOLD :  0.4  **********************
For RESNET : False positive : 60 False negative : 4
For CAM++ : False positive : 52 False negative : 2
For TDN : False positive : 68 False negative : 0
For RESNET : EER Accuracy : 24.067921742340346
For CAM+

In [59]:
test = []

path1 = '23456789876543'
path2 = '12345678987654'
path3 = '12345678937654'



In [61]:
test.append({path1, path2})

In [64]:
if {path2, path1} in test :
    print('yes')

yes
